In [1]:
import pandas as pd 
import numpy as np 
from scipy.stats import entropy
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt

from engine_hms_trainer import *
from engine_hms_model import CustomModel, JobConfig, ModelConfig

import torch
from torch import nn
import torch.nn.functional as F

/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
seed_everything(JobConfig.SEED)

ModelConfig.EPOCHS = 6
ModelConfig.USE_EEG_SPECTROGRAMS = False
ModelConfig.MODEL_BACKBONE = 'tf_efficientnet_b0'
ModelConfig.MODEL_NAME = "ENet_b0_two_stages_xymask"
ModelConfig.AUGMENT = True
ModelConfig.AUGMENTATIONS = ['xy_masking']

hms_predictor = HMSPredictor(JobConfig, ModelConfig)

****************************************************************************************************
Script Start: Sat Mar  9 14:35:30 2024
Initializing HMS Predictor...
Model Name: ENet_b0_two_stages_xymask
Drop Rate: 0.15
Drop Path Rate: 0.25
Augment: True
Augmentations: ['xy_masking']
Enropy Split: 5.5
Device: cuda
Output Dir: ./outputs/
****************************************************************************************************


In [3]:
train_easy, train_hard, all_specs, all_eegs = hms_predictor.load_train_data()

print(train_easy.shape)
print(train_hard.shape)

# check if contain NaN
print(train_easy.isnull().sum().sum())
print(train_hard.isnull().sum().sum())

display(train_easy.head())
print(" ")
display(train_hard.head())

(11999, 14)
(5090, 14)
0
0


,eeg_id,spectrogram_id,min,max,patient_id,target,total_votes,entropy,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,642382,14960202,1008.0,1032.0,5955,Other,2,7.802343,0.0,0.0,0.0,0.0,0.0,1.0
1,751790,618728447,908.0,908.0,38549,GPD,1,7.802343,0.0,0.0,1.0,0.0,0.0,0.0
2,778705,52296320,0.0,0.0,40955,Other,2,7.686820,0.0,0.0,0.0,0.0,0.0,1.0
3,1629671,2036345030,0.0,160.0,37481,Seizure,51,7.619243,1.0,0.0,0.0,0.0,0.0,0.0
4,2061593,320962633,1450.0,1450.0,23828,Other,1,7.802343,0.0,0.0,0.0,0.0,0.0,1.0


,eeg_id,spectrogram_id,min,max,patient_id,target,total_votes,entropy,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,789577333,0.0,16.0,20654,Other,48,3.341757,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333
1,582999,1552638400,0.0,38.0,20230,LPD,154,3.550549,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429
2,1895581,128369999,1138.0,1138.0,47999,Other,13,3.565051,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154
3,2482631,978166025,1902.0,1944.0,20606,Other,105,1.431066,0.000000,0.000000,0.133333,0.066667,0.133333,0.666667
4,2521897,673742515,0.0,4.0,62117,Other,24,1.516203,0.000000,0.000000,0.083333,0.083333,0.333333,0.500000


In [4]:
# Use only half data for fast debugging
# train_easy = train_easy[:len(train_easy)//2]
# train_hard = train_hard[:len(train_hard)//2]

hms_predictor.train_folds(train_easy, train_hard, all_specs, all_eegs)

Fold: 0 First Training


Train:   0%|          | 0/599 [00:00<?, ?batch/s]

Epoch: [1][0/599]Elapsed 1.02s | Loss: 0.8618 Grad: 68081.0781 LR: 4.0000e-06
Epoch: [1][50/599]Elapsed 4.47s | Loss: 0.8274 Grad: 83081.7344 LR: 5.1479e-06
Epoch: [1][100/599]Elapsed 7.88s | Loss: 0.8203 Grad: 68477.1484 LR: 8.5368e-06
Epoch: [1][150/599]Elapsed 11.29s | Loss: 0.8157 Grad: 56965.2031 LR: 1.4005e-05
Epoch: [1][200/599]Elapsed 14.70s | Loss: 0.8114 Grad: 75494.9922 LR: 2.1290e-05
Epoch: [1][250/599]Elapsed 18.11s | Loss: 0.8021 Grad: 75641.4844 LR: 3.0044e-05
Epoch: [1][300/599]Elapsed 21.53s | Loss: 0.7888 Grad: 79959.8438 LR: 3.9848e-05
Epoch: [1][350/599]Elapsed 24.95s | Loss: 0.7756 Grad: 67523.0312 LR: 5.0233e-05
Epoch: [1][400/599]Elapsed 28.37s | Loss: 0.7593 Grad: 73504.3984 LR: 6.0703e-05
Epoch: [1][450/599]Elapsed 31.80s | Loss: 0.7431 Grad: 112829.3672 LR: 7.0757e-05
Epoch: [1][500/599]Elapsed 35.23s | Loss: 0.7288 Grad: 98570.4531 LR: 7.9913e-05
Epoch: [1][550/599]Elapsed 38.65s | Loss: 0.7125 Grad: 114198.7969 LR: 8.7735e-05
Epoch: [1][598/599]Elapsed 41.83

Valid:   0%|          | 0/150 [00:00<?, ?batch/s]

Epoch: [1][0/150]Elapsed 0.08s | Loss: 0.6532
Epoch: [1][50/150]Elapsed 2.54s | Loss: 0.5358
Epoch: [1][100/150]Elapsed 4.99s | Loss: 0.5427


...

Train:   0%|          | 0/254 [00:00<?, ?batch/s]

Epoch: [6][0/254]Elapsed 0.06s | Loss: 0.2531 Grad: 98204.1875 LR: 7.2349e-05
Epoch: [6][50/254]Elapsed 3.62s | Loss: 0.1588 Grad: 67030.6016 LR: 6.9753e-05
Epoch: [6][100/254]Elapsed 7.09s | Loss: 0.1550 Grad: 64899.3008 LR: 6.7093e-05
Epoch: [6][150/254]Elapsed 10.54s | Loss: 0.1560 Grad: 57258.4336 LR: 6.4376e-05
Epoch: [6][200/254]Elapsed 14.05s | Loss: 0.1553 Grad: 55085.1172 LR: 6.1613e-05
Epoch: [6][250/254]Elapsed 17.54s | Loss: 0.1529 Grad: 82547.2422 LR: 5.8812e-05
Epoch: [6][253/254]Elapsed 17.77s | Loss: 0.1528 Grad: 122302.8281 LR: 5.8586e-05


Valid:   0%|          | 0/64 [00:00<?, ?batch/s]

Epoch: [6][0/64]Elapsed 0.05s | Loss: 0.3147
Epoch: [6][50/64]Elapsed 2.55s | Loss: 0.2442


----------------------------------------------------------------------------------------------------
Epoch 6 - Average Train Loss: 0.1528 | Average Valid Loss: 0.2414 | Time: 21.10s
Best model found in epoch 6 | valid loss: 0.2414
CV Result (Stage=2): 0.4424395827373606 (torch) | 0.4424395818829547 (kaggle)
Elapse: 35.17 min 


In [ ]:
dataset = CustomDataset(train_easy, TARGETS, ModelConfig, all_specs, all_eegs, mode='test')

X, y = dataset[0]
print(X.shape, y.shape)

model = CustomModel(ModelConfig, num_classes=6, pretrained=True)
y_pred = model(X.unsqueeze(0))

print(y_pred.shape)

In [ ]:
from kl_divergence import score as kl_score


def calc_kl_div(p, q, criterion):
    
    p = torch.tensor(p.astype(np.float32)).unsqueeze(0)
    q = torch.tensor(q.astype(np.float32)).unsqueeze(0)
    return criterion(F.log_softmax(p, dim=1), q).item()

def calc_kaggle_score(solution, submission):
    solution = solution.to_frame().T
    solution[TARGETS] = solution[TARGETS].astype(np.float32)
    submission = submission.to_frame().T
    submission.columns = ['eeg_id'] + TARGETS
    submission[TARGETS] = submission[TARGETS].astype(np.float32)
    
    return kl_score(solution, submission, 'eeg_id')

In [ ]:
def evaluate_oof(oof_csv_path):
    oof_df = pd.read_csv(oof_csv_path)
    softmax = nn.Softmax(dim=1)
    criterion = nn.KLDivLoss(reduction="batchmean")

    oof_df["kl_loss"] = oof_df.apply(lambda row: 
        calc_kl_div(row[TARGETS_PRED].values, row[TARGETS].values, criterion), axis=1
        )

    kl_loss_all = criterion(
        F.log_softmax(torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32)), dim=1),
        torch.tensor(oof_df[TARGETS].values.astype(np.float32)),
        )

    print(f"KL Loss All: {kl_loss_all}")
    print(f"KL Loss Mean: {oof_df['kl_loss'].mean()}")

    y_pred = oof_df[TARGETS].values.astype(np.float32)
    oof_df[TARGETS_PRED] = softmax(torch.tensor(y_pred)).numpy()

    solution = oof_df[['eeg_id'] + TARGETS].copy()
    submission = oof_df[['eeg_id'] + TARGETS_PRED].copy()
    submission.columns = ['eeg_id'] + TARGETS

    kaggle_score_all = kl_score(solution, submission, 'eeg_id')
    
    oof_df['kaggle_score'] = oof_df.apply(lambda row:
        calc_kaggle_score(row[['eeg_id'] + TARGETS], row[['eeg_id'] + TARGETS_PRED]), axis=1
        )

    print(f"Kaggle Score All: {kaggle_score_all}")
    print(f"Kaggle Score Mean: {oof_df['kaggle_score'].mean()}")

    return oof_df, kl_loss_all, kaggle_score_all


In [ ]:
oof_1, kl_loss_all, kaggle_score_all = evaluate_oof(f"{JobConfig.OUTPUT_DIR}/oof_1.csv")
oof_2, kl_loss_all, kaggle_score_all = evaluate_oof(f"{JobConfig.OUTPUT_DIR}/oof_2.csv")

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(10, 10), sharex=True, sharey=True)

# rows = oof_df.iloc[-len(axes.ravel()):, :]
rows = oof_1.sample(len(axes.ravel()))

for i, (idx, row) in enumerate(rows.iterrows()):

    ax = axes.ravel()[i]
    ax.plot(row[TARGETS].values, label='True')
    ax.plot(row[TARGETS_PRED].values, label='Pred')
    ax.set_title(f"{idx} | {row['target']} | KL: {row['kl_loss']:.4f}")
    ax.set_xticks(range(6))
    ax.set_xticklabels(BRAIN_ACTIVITY)
    ax.grid(True)
    ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
row = oof_1.loc[2619]

min_pred = row[TARGETS_PRED].min()
max_pred = row[TARGETS_PRED].max()

targets_norm = (row[TARGETS] - row[TARGETS].min()) / (row[TARGETS].max() - row[TARGETS].min())

print(targets_norm)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(10, 10), sharex=True, sharey=True)

rows = oof_2.iloc[5:5+len(axes.ravel()), :]
# rows = oof_df.sample(len(axes.ravel()))

for i, (idx, row) in enumerate(rows.iterrows()):

    ax = axes.ravel()[i]
    y_true = row[TARGETS].values
    y_pred = row[TARGETS_PRED].values
    y_norm = (y_true - y_true.min()) / (y_true.max() - y_true.min())

    ax.plot(row[TARGETS].values, label='True')
    ax.plot(row[TARGETS_PRED].values, label='Pred')
    ax.plot(y_norm, "b:", label='True Norm')

    ax.set_title(f"{idx} | {row['target']} | KL: {row['kl_loss']:.4f}")
    ax.set_xticks(range(6))
    ax.set_xticklabels(BRAIN_ACTIVITY)
    ax.grid(True)
    ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
row = oof_2.loc[6]

min_pred = row[TARGETS_PRED].min()
max_pred = row[TARGETS_PRED].max()
print(min_pred, max_pred)

print(row[TARGETS_PRED])

targets_norm = (row[TARGETS] - row[TARGETS].min()) / (row[TARGETS].max() - row[TARGETS].min())

targets_norm = targets_norm / targets_norm.sum()

print(targets_norm)